# 11 — Errors, Exceptions & Debugging Patterns

Goal: Get comfortable with how Python fails, how to read error messages, and how to handle exceptions without turning your code into a try/except soup.

In practice this means:

- Recognising common error types (SyntaxError, TypeError, ValueError, IndexError, KeyError, etc.)
- Reading tracebacks so you know **where** and **why** things blew up
- Using `try` / `except` / `finally` / `raise` in a controlled way
- Adopting a few simple debugging habits (print-style debugging, small repros, isolating functions)

The aim isn’t to avoid errors — it’s to **use them as information** instead of getting stuck or randomly poking at the code.


## 1. Errors vs Exceptions

In Python:

- **Syntax errors** → code is not valid Python. The interpreter refuses to run it.
- **Exceptions** → code is valid, but something went wrong *while running it*.

Examples:

- `SyntaxError` → missing colon, bad indentation, broken string
- `ZeroDivisionError` → divide by zero
- `NameError` → use a variable that doesn’t exist
- `TypeError` → use a value in a way that doesn’t match its type
- `IndexError` → list index out of range
- `KeyError` → missing key in a dict
- `ValueError` → value has the right type, but is invalid

We’ll mostly focus on **exceptions**, because that’s what you’ll hit in normal code.


In [ ]:
# We'll trigger a few common exceptions on purpose.
# Uncomment each block one at a time to see the traceback.


# 1) ZeroDivisionError
# result = 10 / 0


# 2) NameError
# print(unknown_variable)


# 3) TypeError
# total = "3" + 4


# 4) IndexError
# nums = [1, 2, 3]
# print(nums[5])


# 5) KeyError
# d = {"a": 1}
# print(d["b"])


# 6) ValueError
# int("not_an_int")


## 2. Reading a Traceback

When an exception happens, Python prints a **traceback**:

- The **last line** tells you:
  - the *type* of exception (`ValueError`, `KeyError`, etc.)
  - a short message
- The lines above show **where** it happened: file, line number, and code context.

Example pattern:

```text
Traceback (most recent call last):
  File "script.py", line 10, in <module>
    int("abc")
ValueError: invalid literal for int() with base 10: 'abc'
```
Mental flow:

1. croll to the bottom line → what kind of error?

2. Look at the last stack frame → which line in your code caused it?

3. Use that as your starting point instead of randomly guessing.

In [ ]:
def parse_int(s):
    return int(s)

def demo():
    return parse_int("abc")

# Uncomment to see the traceback.
# demo()

## 3. Handling Exceptions with `try` / `except`

Sometimes you **expect** something might fail and want to handle it gracefully.

Basic structure:

```python
try:
    risky_operation()
except SomeError:
    handle_the_problem()
```
Control flow:

- Python runs the `try` block.

- If no exception → `except` is skipped.

- If an exception of the specified type happens → jump to `except` and run that block.

In [ ]:
def safe_int(s):
    try:
        return int(s)
    except ValueError:
        print(f"Could not convert {s!r} to int")
        return None

print(safe_int("123"))
print(safe_int("xyz"))

## 4. Catching Specific Exceptions

Avoid catching **everything** with a bare `except:` unless you *really* mean it.

Instead, catch what you expect:

```python
try:
    value = int(s)
except ValueError:
    ...
```
You can also catch multiple types:
```python
try:
    ...
except (TypeError, ValueError):
    ...
```
Catching everything makes debugging harder, because it hides bugs you didn’t expect.


In [ ]:
def divide(a, b):
    try:
        return a / b
    except ZeroDivisionError:
        print("Cannot divide by zero!")
        return None

print(divide(10, 2))
print(divide(10, 0))

## 5. `finally` — Code That Always Runs

`finally` is for code that should run **no matter what**:

```python
try:
    ...
except SomeError:
    ...
finally:
    # always runs
    cleanup()
```
Typical uses:

- closing files

- releasing resources

- logging that something finished (success or fail)

In [ ]:
def process_item(item):
    print("Processing:", item)
    if item == "bad":
        raise ValueError("Bad item!")
    print("Done processing:", item)

items = ["ok", "bad", "ok again"]

for item in items:
    try:
        process_item(item)
    except ValueError as e:
        print("Caught error:", e)
    finally:
        print("Cleaning up...\n")

## 6. Raising Your Own Exceptions

Sometimes you detect a bad situation and want to signal it explicitly.

Use `raise`:

```python
def sqrt(x):
    if x < 0:
        raise ValueError("x must be non-negative")
    return x ** 0.5
```
Raising exceptions is better than silently returning nonsense.<br>
In ML code, you'd rather fail loudly than quietly continue with invalid data.

In [ ]:
def clipped_mean(values):
    if not values:
        raise ValueError("values must not be empty")
    return sum(values) / len(values)

print(clipped_mean([1, 2, 3]))

# Uncomment to see the exception:
# clipped_mean([])

## 7. Simple Debugging Habits

Instead of "staring at it until it works", use structure.

Some patterns:

### 1. Print-style debugging

Add temporary prints to inspect values:

```python
print("x =", x)
print("shape =", x.shape)
```
### 2. Reduce to a minimal example

Copy the failing bit into a small script or cell with fake data that reproduces the error. <br>
If you can reproduce the bug with 5–10 lines, it becomes much easier to reason about.

### 3. Check assumptions with `assert`
```python
assert len(values) > 0, "values must not be empty"
```
If the assertion fails, you get a clear failure at the right point.

### 4. One change at a time

When fixing something, change a small piece, run it, then move on. <br>
Avoid editing 20 lines and then not knowing which change broke or fixed it.

In [ ]:
def normalize(values):
    assert len(values) > 0, "values must not be empty"
    total = sum(values)
    return [v / total for v in values]

print(normalize([1, 1, 2]))

## 8. ML-Flavoured Errors You’ll Actually See

Common patterns:

- **Shape mismatches**  
```python
  ValueError: operands could not be broadcast together with shapes (3,) (4,)
```
→ Often means your vectors/matrices don’t match in size.

- KeyErrors in metric dicts / configs

```python
KeyError: 'accuracy'
```
→ You're trying to access a key that isn’t there.

- NaNs or infs in loss
```python
RuntimeWarning: invalid value encountered in ...
```
→ Often due to `log(0)`, division by zero, huge learning rate, or exploding values.

Debugging flow:

1. Read the traceback bottom line.

2. Find the line in your code.

3. Print / inspect the key variables right before that line.

4. Shrink the example if needed until it’s obvious.

# 11 — Exercises (Errors, Exceptions & Debugging)

### Exercise 1 — Identify the Exception

Without running, predict what exception each will raise:

```python
1)  "3" + 4

2)  nums = [1, 2, 3]
    nums[10]

3)  d = {"x": 1}
    d["y"]

4)  int("3.14")
```
Then run them one by one (inside a safe test cell) to check.

In [ ]:
# Excercise 1
#1)  
# "3" + 4

#2)  
# nums = [1, 2, 3]
#   nums[10]

#3)  
# d = {"x": 1}
#d["y"]

#4)  
# int("3.14")

### Exercise 2 — Safe Division

Write a function:

```python
def safe_divide(a, b):
    ...
```
Rules:
- If `b` is 0, print `"Cannot divide by zero"` and return `None`.

- Otherwise, return `a / b`.

Use `try / except ZeroDivisionError`.

In [ ]:
# Excercise 2
def safe_divide(a, b):
    ...

### Exercise 3 — Parsing Integers Safely

Write:
```python
def parse_int_list(strings):
    ...
```
Given a list of strings, convert them to ints:

- If a value cannot be converted, skip it but do not crash.

- Return the list of successfully parsed ints.

Example:
```python
parse_int_list(["10", "x", "3"])  # [10, 3]
```

In [ ]:
# Excercise 3
def parse_int_list(strings):
    ...

parse_int_list(["10", "x", "3"])  # [10, 3]

### Exercise 4 — Custom `raise`

Write:
```python
def mean_strict(values):
    ...
```
Rules:

- If `values` is empty, `raise ValueError("values must not be empty")`.

- Otherwise, return the mean.

Show it working for a normal list and raising for `[]`.

In [ ]:
# Excercise 4
def mean_strict(values):
    ...

### Exercise 5 — Debugging with Assertions

You’re given this buggy function:
```python
def normalize_non_negative(values):
    total = sum(values)
    return [v / total for v in values]
```
Add an assertion at the top that:

- raises if `values` is empty

- raises if any value is negative

Then test with:

- `[1, 2, 3]` (should work)

- `[]`

- `[1, -1, 2]`

and see the assertion message.

In [ ]:
# Excercise 5 
def normalize_non_negative(values):
    total = sum(values)
    return [v / total for v in values]